- Maximize growth of each AGORA2 model
- constrain to 75% of growth
- catalogue the produced and consumed metabolites
- generate a massive table of the produced/consumed metabolites
-- Maybe create a function that automatically conducts this analysis with a parameterized biomass percent of maximum limit and the type of reactions to be examined

In [16]:
from pandas import read_csv
from glob import glob

bigg_db = read_csv("bigg_metabolite_mapping.tsv", sep="\t").set_index("universal_bigg_id")
display(bigg_db)
# print(list(bigg_db.index))

,bigg_id,name,model_list,database_links,old_bigg_ids
universal_bigg_id,,,,,
12dgr120,12dgr120_c,"1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)",iEC1364_W; iEC1349_Crooks; iEC1356_Bl21DE3; iM...,MetaNetX (MNX) Chemical: http://identifiers.or...,12dgr120; 12dgr120[c]; 12dgr120_c; _12dgr120_c
12dgr140,12dgr140_c,"1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)",iECNA114_1301; iECSE_1348; iECO111_1330; iECOK...,MetaNetX (MNX) Chemical: http://identifiers.or...,12dgr140; 12dgr140[c]; 12dgr140_c; _12dgr140_c
12dgr180,12dgr180_c,"1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)",iECB_1328; iECDH10B_1368; iEcE24377_1341; iECD...,MetaNetX (MNX) Chemical: http://identifiers.or...,12dgr180; 12dgr180[c]; 12dgr180_c; _12dgr180_c
14glucan,14glucan_c,"1,4-alpha-D-glucan",iSFxv_1172; iUTI89_1310; iSSON_1240; iSbBS512_...,BioCyc: http://identifiers.org/biocyc/META:1-4...,14glucan; 14glucan_c
15dap,15dap_c,"1,5-Diaminopentane",iECUMN_1333; iLF82_1304; iETEC_1333; iECSF_132...,KEGG Compound: http://identifiers.org/kegg.com...,15dap; 15dap[c]; 15dap_c
...,...,...,...,...,...
glp,glp_c,Glycylpeptide C4H7N2O3R,iND750; iMM904,KEGG Compound: http://identifiers.org/kegg.com...,glp; glp_c
glu__L,glu__L_x,L-Glutamate,iMM904; iND750,Reactome Compound: http://identifiers.org/reac...,glu_DASH_L_x; glu_L_x; glu__L
glutrna,glutrna_m,L-Glutamyl-tRNA(Glu),iND750; iMM904; iIS312; iIS312_Epimastigote; i...,Reactome Compound: http://identifiers.org/reac...,glutrna; glutrna_m


In [17]:
def levenshtein_distance(vals):
    s1, s2 = vals
    if len(s1) < len(s2):
        return levenshtein_distance((s2, s1))

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]


print(levenshtein_distance('26dap_M', '26dap__M'))

TypeError: levenshtein_distance() takes 1 positional argument but 2 were given

In [ ]:
from cobra.io import read_sbml_model
from pandas import read_csv
from glob import glob

bigg_db = read_csv("bigg_metabolite_mapping.tsv", sep="\t").set_index("universal_bigg_id")

base_fvas = {}
for fva in glob("AGORA2_fva/*"):
    model_fva = read_csv(fva).set_index("Unnamed: 0")
    organism_name = fva.replace("M_", "").replace("_fva", "").replace(".csv", "").split("/")[1]
    # org_model = read_sbml_model(f"../AGORA2_models/{organism_name}.xml")
    
    base_fvas[organism_name] = {}
    for index, row in model_fva.iterrows():
        if "EX_" not in index:  continue
        base_metName = index.replace("EX_", "").split("(")[0]
        # if base_metName not in bigg_db.index:   print(f"the {base_metName} met is not defined the BiGG database")  ;  continue
        if base_metName in bigg_db.index:  val = bigg_db.loc[base_metName, "name"]
        else:
            # val = org_model.metabolites.get_by_id(base_metName).name
            similarityCount = dict(zip(bigg_db.index, list(map(levenshtein_distance, [(base_metName, index) for index in bigg_db.index]))))
            best_matchingID = min(similarityCount, key=similarityCount.get)
            print(best_matchingID, "\tfrom\t", base_metName)
            val = bigg_db.loc[best_matchingID, "name"]
        if row["maximum"] > 0:
            if "produced" in base_fvas[organism_name]:   base_fvas[organism_name]["produced"].append(val)
            else: base_fvas[organism_name]["produced"] = [val]
        if row["minimum"] < 0:
            if "consumed" in base_fvas[organism_name]:   base_fvas[organism_name]["consumed"].append(val)
            else: base_fvas[organism_name]["consumed"] = [val]

KeyboardInterrupt: 

In [ ]:
from json import dump

with open("AGORA2_EX_consumption_production.json", "w") as jsonOut:
    dump(base_fvas, jsonOut, indent=3)

In [ ]:
from pandas import set_option

set_option("display.max_columns", None)
display(total_df)

,Unnamed: 0,minimum,maximum
0,12DGR180ti,1.928097,1.928097
1,23DHMPO,-2.209403,-2.209403
2,23PDE2,0.000000,0.000000
3,23PDE4,0.000000,0.000000
4,23PDE7,0.000000,0.000000
...,...,...,...
1109,r2137,0.181550,0.181550
1110,rtranscription,58.630618,58.630618
1111,sink_PGPm1[c],-0.317661,-0.317661
1112,bio1,58.630618,58.630618


# Compute the FVA for all AGORA2 models

In [19]:
%run ../ModelSEEDpy/modelseedpy/core/msmodelutl.py
from optlang import Constraint, Objective
from cobra.io import read_sbml_model
from json import load, dump
from glob import glob

computed_fvas = {}
for fva_path in glob("AGORA2_flux_exchanges*.json"):
    with open(fva_path, "r") as jsonIn:   saved_fva = load(jsonIn)
    computed_fvas.update(saved_fva)

print(len(glob("AGORA2_flux_exchanges*.json")))

fva_dic = {}
model_consProd = {}
hundreds_count, count = len(glob("AGORA2_flux_exchanges*.json")), 0
for mdl_path in glob("../AGORA2_models/*.xml"):
    if "M_"+mdl_path.split("/")[-1].replace(".xml", "") in computed_fvas: print(f"DONE:  {mdl_path}") ; continue
    try:   util = MSModelUtil(read_sbml_model(mdl_path))
    except Exception as e: print(f"FAIL:  {mdl_path}") ; continue
    print(util.model.id)
    count += 1
    util.add_minimal_objective_cons(min_value=util.model.slim_optimize()*.75)
    model_consProd[util.model.id] = {"production": [], "consumption": []}
    fva_dic[util.model.id] = {}
    for ex in util.exchange_list():
        ex_cpd= util.model.metabolites.get_by_id([*ex.metabolites][0].id)
        # production
        util.set_objective_from_target_reaction(ex, minimize=False)
        prod_obj_val = util.model.slim_optimize()
        if prod_obj_val > 0:   model_consProd[util.model.id]["production"].append(ex_cpd.name)
        # consumption
        util.set_objective_from_target_reaction(ex, minimize=True)
        cons_obj_val = util.model.slim_optimize()
        if cons_obj_val < 0:   model_consProd[util.model.id]["consumption"].append(ex_cpd.name)
        fva_dic[util.model.id][ex.id] = (cons_obj_val, prod_obj_val)
    if count%100 == 0:
        with open(f"AGORA2_flux_exchanges{hundreds_count}.json", "w") as jsonOut:
            dump(model_consProd, jsonOut, indent=3)
        hundreds_count += 1
        model_consProd = {}


with open(f"AGORA2_flux_exchanges{hundreds_count}.json", "w") as jsonOut:
    dump(model_consProd, jsonOut, indent=3)

63
DONE:  ../AGORA2_models/Klebsiella_aerogenes_CAV1320.xml
DONE:  ../AGORA2_models/Clostridium_isatidis_DSM_15098.xml
DONE:  ../AGORA2_models/Schwartzia_succinivorans_DSM_10502.xml
DONE:  ../AGORA2_models/Mycobacterium_avium_subsp_paratuberculosis_JQ6.xml
DONE:  ../AGORA2_models/Lactobacillus_salivarius_GJ_24.xml
DONE:  ../AGORA2_models/Streptococcus_mutans_SF12.xml
DONE:  ../AGORA2_models/Salmonella_enterica_subsp_enterica_serovar_Agona_str_63_H_87.xml
DONE:  ../AGORA2_models/Clostridium_botulinum_D_str_1873.xml
DONE:  ../AGORA2_models/Staphylococcus_aureus_subsp_aureus_CIG2018.xml
DONE:  ../AGORA2_models/Enterococcus_faecium_VAN_219.xml
DONE:  ../AGORA2_models/Clostridium_symbiosum_ATCC_14940.xml
DONE:  ../AGORA2_models/Shigella_flexneri_K_304.xml
DONE:  ../AGORA2_models/Caecibacter_massiliensis_DSM_100188.xml
DONE:  ../AGORA2_models/Lactobacillus_amylovorus_GRL1118.xml
DONE:  ../AGORA2_models/Streptococcus_sanguinis_SK115.xml
DONE:  ../AGORA2_models/Ureaplasma_urealyticum_serovar_2

## combine all exported FVA partitions

In [20]:
from json import load, dump

computed_fvas = {}
for fva_path in glob("AGORA2_flux_exchanges*.json"):
    with open(fva_path, "r") as jsonIn:   saved_fva = load(jsonIn)
    computed_fvas.update(saved_fva)

with open("AGORA2_total_flux_exchanges.json", "w") as jsonOut:   dump(computed_fvas, jsonOut, indent=3)

## determine the frequency of compounds excreted

In [21]:
from collections import Counter

with open("AGORA2_total_flux_exchanges.json", "r") as jsonIn:   total_fva = load(jsonIn)
produced_freq = Counter([prod for content in total_fva.values() for prod in content["production"]])
consumed_freq = Counter([cons for content in total_fva.values() for cons in content["consumption"]])

with open("AGORA2_produced_freq.json", "w") as jsonOut:
    dump(dict(sorted(produced_freq.items(), key=lambda item: item[1], reverse=True)), jsonOut, indent=3)
with open("AGORA2_consumed_freq.json", "w") as jsonOut:
    dump(dict(sorted(consumed_freq.items(), key=lambda item: item[1], reverse=True)), jsonOut, indent=3)

In [ ]:
from pandas import DataFrame
from json import dump

flux_ranges = DataFrame(fva_dic)
display(flux_ranges)
flux_ranges.to_csv("AGORA2_flux_exchanges.csv")

with open("AGORA2_flux_exchanges.json", "w") as jsonOut:  dump(model_consProd, jsonOut, indent=3)

""
